In [1]:
#imported libs
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.utils import get_file
import os
from sklearn.preprocessing import StandardScaler

In [2]:
#dataset_dir  = r"E:\APPS\PythonDataSets\caltech\caltech-101\101_ObjectCategories\101_ObjectCategories"
dataset_dir = r"E:\APPS\PythonDataSets\caltech\caltech-101\101_ObjectCategories\filtered_preprocessed_categories"



# Parameters
batch_size = 32
image_size = (64, 128)

# Load the training and validation datasets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)


def dataset_to_numpy(dataset):
    """
    Convert a tf.data.Dataset into NumPy arrays for features and labels.
    Args:
        dataset: A tf.data.Dataset object.
    Returns:
        X: Numpy array of features (images).
        y: Numpy array of labels.
    """
    X = []
    y = []
    for images, labels in dataset:
        X.append(images.numpy())
        y.append(labels.numpy())
    return np.concatenate(X, axis=0), np.concatenate(y, axis=0)

# Convert the train and validation datasets to NumPy arrays
X_train, y_train = dataset_to_numpy(train_dataset)
X_test, y_test = dataset_to_numpy(val_dataset)

X_test =[cv2.resize(img.astype(np.uint8), (64, 128)) for img in X_test]
X_train=[cv2.resize(img.astype(np.uint8), (64, 128)) for img in X_train]


Found 9145 files belonging to 101 classes.
Using 7316 files for training.
Found 9145 files belonging to 101 classes.
Using 1829 files for validation.


In [ ]:
#SVM def
import cv2
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

def SVM(X_train, y_train,X_test,y_test):

    # Step 4: Train LinearSVC with Grid Search for Hyperparameter Tuning
    param_grid = {
        'C': [0.1,10],  # Regularization parameter
    }
    svm = LinearSVC(max_iter=10000)  # Linear SVM for multiclass classification dual='auto',
    grid_search = GridSearchCV(svm, param_grid, cv=3, verbose=2 , n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Best Model
    best_svm = grid_search.best_estimator_
    print("Best Parameters:", grid_search.best_params_)

    # Step 5: Evaluate Model
    y_pred = best_svm.predict(X_test)

    # Classification Report    
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))#,zero_division=1

In [ ]:
#Color Histogram Extraction def
def extract_color_histogram(image, bins=(8, 8, 8)):
    # Calculate the 3D histogram for the HSV channels
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    # Normalize the histogram to ensure invariance to lighting changes
    hist = cv2.normalize(hist, hist).flatten()

    return hist

In [4]:
#HOG def
def extract_hog_features(image):
    # HOG parameters
    winSize = (64, 128)
    blockSize = (16, 16)
    blockStride = (8, 8)
    cellSize = (8, 8)
    nbins = 9
    
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    hog_features = hog.compute(image)
     
    return hog_features

In [13]:
#LBP def
from skimage.feature import local_binary_pattern

def extract_lbp_features(image, num_points=32, radius=8):
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    grid_size = (8, 8)  # Divide image into a 8x8 grid for histograms
     # Compute LBP
    lbp = local_binary_pattern(gray_img, num_points, radius, method="uniform")
    h, w = lbp.shape
    
    # Divide the image into grids and compute histograms
    grid_h, grid_w = h // grid_size[0], w // grid_size[1]
    histograms = []
    
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            grid = lbp[i * grid_h:(i + 1) * grid_h, j * grid_w:(j + 1) * grid_w]
            hist, _ = np.histogram(grid, bins=np.arange(0, num_points + 3), density=True)
            histograms.append(hist)
            
    return np.concatenate(histograms)

In [14]:
# Step 1: Extract LBP features for train and test
lbp_features_train = np.array([extract_lbp_features(image) for image in X_train])
lbp_features_test  = np.array([extract_lbp_features(image) for image in X_test])

In [5]:
# Step 2: Extract HOG features for train and test
hog_features_train = np.array([extract_hog_features(image) for image in X_train])
hog_features_test  = np.array([extract_hog_features(image) for image in X_test])
scaler = StandardScaler()
hog_features_train = scaler.fit_transform(hog_features_train)
hog_features_test = scaler.transform(hog_features_test)

In [24]:
# Step 3: Extract Color Histogram features for train and test
clhg_features_train = np.array([extract_color_histogram(image) for image in X_train])
clhg_features_test  = np.array([extract_color_histogram(image) for image in X_test])

In [22]:
SVM(lbp_features_train,y_train,lbp_features_test,y_test)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best Parameters: {'C': 1}
Accuracy: 0.47785675232367414
              precision    recall  f1-score   support

           0       0.15      0.18      0.17        92
           1       0.83      0.95      0.89       175
           2       0.70      0.95      0.81        44
           3       0.89      0.97      0.93       176
           4       0.53      0.73      0.62        11
           5       0.80      0.93      0.86       162
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00        17
          10       0.12      0.14      0.13         7
          11       1.00      0.00      0.00         9
          12       0.20      0.24      0.22        25
          13       0.37      0.37      0.37        19
          14       0.33      0.43      0.38         7
          15       0

In [6]:
SVM(hog_features_train,y_train,hog_features_test,y_test)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


KeyboardInterrupt: 

In [27]:
SVM(clhg_features_train,y_train,clhg_features_test,y_test)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Best Parameters: {'C': 10}
Accuracy: 0.32640787315472936
              precision    recall  f1-score   support

           0       0.17      0.10      0.12        92
           1       0.62      0.95      0.75       175
           2       0.65      0.80      0.71        44
           3       0.32      0.88      0.47       176
           4       0.12      0.27      0.17        11
           5       0.44      0.73      0.55       162
           6       1.00      0.00      0.00         7
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00        10
           9       1.00      0.00      0.00        17
          10       1.00      0.00      0.00         7
          11       0.00      0.00      0.00         9
          12       0.29      0.08      0.12        25
          13       0.09      0.16      0.12        19
          14       0.00      0.00      0.00         7
          15       